In [20]:
# Read the Excel file
file_path = "/Users/chaitanyakartik/Downloads/main_account_statement.xlsx"
print(f"Reading Excel file: {file_path}")

# Get the sheet names first
xl = pd.ExcelFile(file_path)
sheet_names = xl.sheet_names

column_names = {}

# Read each sheet and get column names
for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    column_names[sheet] = list(df.columns)
    print(f"\nSheet: {sheet}")
    print(f"Number of columns: {len(df.columns)}")
    print(f"Column names: {df.columns.tolist()}")
    print(f"Sample data (first 2 rows):")
    print(df.head(2))

print("Column names:", column_names)
    


Reading Excel file: /Users/chaitanyakartik/Downloads/main_account_statement.xlsx

Sheet: Table 1
Number of columns: 9
Column names: ['Tran Date', 'Chq No', 'Unnamed: 2', 'Unnamed: 3', 'Particulars', 'Debit', 'Credit', 'Balance', 'Init. Br']
Sample data (first 2 rows):
   Tran Date  Chq No  Unnamed: 2  Unnamed: 3  \
0 2025-08-27     NaN         NaN         NaN   
1 2025-08-27     NaN         NaN         NaN   

                                         Particulars  Debit  Credit  Balance  \
0  UPI/P2A/523955727509/N S MURAL/AXIS\nBANK/Sent...    NaN   500.0   544.56   
1  UPI/P2M/523965745423/Kanti Sweets-SNS Arca/Sen...   35.0     NaN   509.56   

   Init. Br  
0      52.0  
1      52.0  
Column names: {'Table 1': ['Tran Date', 'Chq No', 'Unnamed: 2', 'Unnamed: 3', 'Particulars', 'Debit', 'Credit', 'Balance', 'Init. Br']}


In [21]:
# Keep only the specified columns
columns_to_keep = ['Tran Date', 'Particulars', 'Debit', 'Credit', 'Balance']
df = df[columns_to_keep]
df['Category'] = ''

# Fill all NaN values with 0
df = df.fillna(0)

# Display the first few rows to verify
df.head()

,Tran Date,Particulars,Debit,Credit,Balance,Category
0,2025-08-27 00:00:00,UPI/P2A/523955727509/N S MURAL/AXIS\nBANK/Sent...,0.0,500.0,544.56,
1,2025-08-27 00:00:00,UPI/P2M/523965745423/Kanti Sweets-SNS Arca/Sen...,35.0,0.0,509.56,
2,2025-08-27 00:00:00,UPI/P2M/523916293245/Swiggy Limited /Pay...,233.0,0.0,276.56,
3,2025-08-28 00:00:00,UPI/P2M/524046624617/Chethana M /Sen...,28.0,0.0,248.56,
4,2025-08-28 00:00:00,UPI/P2M/524036618029/BharatPe Merchant\n/Pay t...,10.0,0.0,238.56,


In [37]:
# Define travel-related strings to search for
Travel_List = ['BMTC BUS', 'BUS', 'BUS TICKET', 'BUS FARE', 'BUS CHARGE', "Metro"]

# Create a regex pattern with alternation of all travel keywords
import re
travel_pattern = '|'.join(Travel_List)

# Fill in 'Travel' category where Particulars contains any of these travel keywords
df.loc[df['Particulars'].astype(str).str.contains(travel_pattern, case=False, na=False, regex=True), 'Category'] = 'Travel'

# Define pattern for Karnataka vehicle registration numbers
import re
vehicle_pattern = r'KA\d{2}[A-Z]{2}\d{4}'

# Find rows matching the vehicle registration pattern
vehicle_mask = df['Particulars'].astype(str).str.contains(vehicle_pattern, case=True, na=False, regex=True)

# Add these to the Travel category
df.loc[vehicle_mask, 'Category'] = 'Travel'

# Display the first few rows to see the changes
df.head()

# Check how many rows were categorized as 'Travel'
travel_count = (df['Category'] == 'Travel').sum()
print(f"Number of transactions categorized as 'Travel': {travel_count}")

Number of transactions categorized as 'Travel': 47


In [45]:
hp_mask = df['Particulars'].astype(str).str.contains(r'\bHP\b', case=True, na=False, regex=True)
df.loc[hp_mask, 'Category'] = 'Fuel'
fuel_count = (df['Category'] == 'Fuel').sum()
print(f"Number of transactions categorized as 'Fuel': {fuel_count}")


Number of transactions categorized as 'Fuel': 1


In [48]:
# Define grocery-related strings to search for
grocery_keywords = ['VILLAGE MARKET', 'Swiggy Limited', 'Family mall', 'HANUMESH']

# Create a regex pattern with alternation of all keywords
import re
pattern = '|'.join(grocery_keywords)

# Fill in 'groceries' category where Particulars contains any of these keywords
df.loc[df['Particulars'].astype(str).str.contains(pattern, case=False, na=False, regex=True), 'Category'] = 'groceries'

# Display the first few rows to see the changes
df.head()

# Check how many rows were categorized as 'groceries'
groceries_count = (df['Category'] == 'groceries').sum()
print(f"Number of transactions categorized as 'groceries': {groceries_count}")

Number of transactions categorized as 'groceries': 6


In [55]:
# Define outside food related strings to search for
outside_food_list = ['Biryani', 'coffee day', 'cafe', 'foods', 'Pani Puri', 'Biriyani','RESTAURANT', 'MAGICPIN']

# Create a regex pattern with alternation of all outside food keywords
import re
food_pattern = '|'.join(outside_food_list)

# Fill in 'Outside Food' category where Particulars contains any of these food keywords
df.loc[df['Particulars'].astype(str).str.contains(food_pattern, case=False, na=False, regex=True), 'Category'] = 'Outside Food'

# Display the first few rows to see the changes
df.head()

# Check how many rows were categorized as 'Outside Food'
food_count = (df['Category'] == 'Outside Food').sum()
print(f"Number of transactions categorized as 'Outside Food': {food_count}")

Number of transactions categorized as 'Outside Food': 11


In [50]:
# Define condition for Order Food category
# First create a mask for "swiggy"
swiggy_mask = df['Particulars'].astype(str).str.contains('swiggy', case=False, na=False)
# Then create a mask for "swiggy limited"
swiggy_limited_mask = df['Particulars'].astype(str).str.contains('swiggy limited', case=False, na=False)
# Combine the masks to get entries with "swiggy" but not "swiggy limited"
order_food_mask = swiggy_mask & ~swiggy_limited_mask

# Apply the category
df.loc[order_food_mask, 'Category'] = 'Order Food'

# Check how many rows were categorized as 'Order Food'
order_food_count = (df['Category'] == 'Order Food').sum()
print(f"Number of transactions categorized as 'Order Food': {order_food_count}")

Number of transactions categorized as 'Order Food': 4


In [51]:
# Make a copy of the DataFrame to avoid modifying the original
df_export = df.copy()

# Convert the 'Tran Date' column to string if it's not already
if pd.api.types.is_datetime64_any_dtype(df_export['Tran Date']):
    df_export['Tran Date'] = df_export['Tran Date'].dt.strftime('%Y-%m-%d')
else:
    df_export['Tran Date'] = df_export['Tran Date'].astype(str)

# Define output file path
output_path = "/Users/chaitanyakartik/Downloads/categorized_transactions.xlsx"

# Export DataFrame to Excel
df_export.to_excel(output_path, index=False)

print(f"DataFrame exported successfully to: {output_path}")

DataFrame exported successfully to: /Users/chaitanyakartik/Downloads/categorized_transactions.xlsx


In [52]:
df.head()

,Tran Date,Particulars,Debit,Credit,Balance,Category
0,2025-08-27 00:00:00,UPI/P2A/523955727509/N S MURAL/AXIS\nBANK/Sent...,0.0,500.0,544.56,
1,2025-08-27 00:00:00,UPI/P2M/523965745423/Kanti Sweets-SNS Arca/Sen...,35.0,0.0,509.56,
2,2025-08-27 00:00:00,UPI/P2M/523916293245/Swiggy Limited /Pay...,233.0,0.0,276.56,groceries
3,2025-08-28 00:00:00,UPI/P2M/524046624617/Chethana M /Sen...,28.0,0.0,248.56,
4,2025-08-28 00:00:00,UPI/P2M/524036618029/BharatPe Merchant\n/Pay t...,10.0,0.0,238.56,


In [56]:
# Calculate sum of debits by category
category_spending = df.groupby('Category')['Debit'].sum().reset_index()

# Sort categories by spending (highest to lowest)
category_spending = category_spending.sort_values('Debit', ascending=False)

# Focus on the specific categories requested
target_categories = ['Outside Food', 'Order Food', 'groceries', 'Travel']
specific_spending = category_spending[category_spending['Category'].isin(target_categories)]
petrol_spending = category_spending[category_spending['Category'] == 'Petrol']['Debit'].sum()

print("Spending by Category:")
for index, row in specific_spending.iterrows():
    print(f"{row['Category']}: {row['Debit']:.2f}")

# Calculate the total for these specific categories
total_spending = specific_spending['Debit'].sum()
print(f"\nTotal spending on food, groceries, and travel: {total_spending:.2f}")

# Calculate the percentage of spending on these categories compared to total debits
total_debits = df['Debit'].sum()
percentage = (total_spending / total_debits) * 100 if total_debits > 0 else 0
print(f"This represents {percentage:.2f}% of total debits ({total_debits:.2f})")

Spending by Category:
Travel: 1650.00
Outside Food: 1387.11
groceries: 1043.00
Order Food: 812.00

Total spending on food, groceries, and travel: 4892.11
This represents 4.84% of total debits (101092.18)
